# Intake für das Data-Engineering

Intake unterstützt das Data-Engineering beim Bereitstellen von Daten und bei der Spezifikation der Datenquellen, der Verteilung der Daten, der Parametrisierung der Benutzeroptionen etc. Dies erleichtert im anschließenden Data-Engineering das einfache Erschließen der Daten, da die möglichen Optionen bereits im Katalog angegeben sind.

In [1]:
import hvplot.pandas
import intake


intake.output_notebook()

Intake-Datensätze werden mit sog. Treibern geladen, von denen einige schon mit dem Intake-Paket mitkommen, andere jedoch als [Plugin](https://intake.readthedocs.io/en/latest/plugin-directory.html) nachgeladen werden müssen. Die verfügbaren Treiber könnt ihr euch folgendermaßen anzeigen lassen:

In [2]:
list(intake.registry)

['parquet',
 'alias',
 'catalog',
 'csv',
 'intake_remote',
 'json',
 'jsonl',
 'ndzarr',
 'numpy',
 'textfiles',
 'tiled',
 'tiled_cat',
 'yaml_file_cat',
 'yaml_files_cat',
 'zarr_cat']

Jedem dieser Treiber ist eine `intake.open_*`-Funktion zugeordnet. Es ist auch möglich, auf Treiber mit dem vollständig qualifizierten Namen (z.B. `package.submodule.DriverClass`) zu verweisen. Im folgenden Beispiel konzentrieren wir uns jedoch auf den `csv`-Treiber, der in der Standard-Intake-Installation enthalten ist.

Im Allgemeinen besteht der erste Schritt zum Schreiben eines Katalogeintrags darin, die entsprechende `open_*` Funktion zum Erstellen eines DataSource-Objekts zu verwenden:

In [3]:
source = intake.open_csv(
    "https://timeseries.weebly.com/uploads/2/1/0/8/21086414/sea_ice.csv"
)

Die obige Angabe hat nun ein DataSource-Objekt erstellt, aber noch nicht überprüft, ob tatsächlich auf die Daten zugegriffen werden kann. Um zu testen, ob das Laden wirklich erfolgreich war, kann die Quelle selbst erschlossen (`source.discover`) oder gelesen (`source.read`) werden:

In [4]:
source.discover()

{'dtype': {'Time': 'object', 'Arctic': 'float64', 'Antarctica': 'float64'},
 'shape': (None, 3),
 'npartitions': 1,
 'metadata': {}}

In [5]:
df = source.read()
df.head()

,Time,Arctic,Antarctica
0,1990M01,12.72,3.27
1,1990M02,13.33,2.15
2,1990M03,13.44,2.71
3,1990M04,12.16,5.10
4,1990M05,10.84,7.37


Nachdem wir festgestellt haben, das sich die Daten wie gewünscht laden lassen, wollen wir uns die Daten visuell erschließen:

In [6]:
df.hvplot(
    kind="line", x="Time", y=["Arctic", "Antarctica"], width=700, height=500
)

:NdOverlay   [Variable]
   :Curve   [Time]   (value)

Wir können nun eine Quelle korrekt laden und erhalten auch eine grafische Ausgabe zur Erschließung der Daten. Dieses Rezept können wir uns nun in der YAML-Syntax ausgeben lassen mit:

In [7]:
print(source.yaml())

sources:
  csv:
    args:
      urlpath: https://timeseries.weebly.com/uploads/2/1/0/8/21086414/sea_ice.csv
    description: ''
    driver: intake.source.csv.CSVSource
    metadata: {}



Schließlich können wir eine YAML-Datei erstellen, die dieses Rezept mit einer zusätzlichen Beschreibung und dem getesteten Diagramm enthält:

In [8]:
%%writefile sea.yaml
sources:
    sea_ice:
      args:
        urlpath: "https://timeseries.weebly.com/uploads/2/1/0/8/21086414/sea_ice.csv"
      description: "Polar sea ice cover"
      driver: csv
      metadata:
        plots:
          basic:
            kind: line
            x: Time
            y: [Arctic, Antarctica]
            width: 700
            height: 500

Overwriting sea.yaml


Um zu überprüfen, ob die YAML-Datei auch funktioniert, können wir sie erneut laden und versuchen, damit zu arbeiten:

In [9]:
cat = intake.open_catalog("sea.yaml")

In [10]:
cat.sea_ice.plot.basic()

:NdOverlay   [Variable]
   :Curve   [Time]   (value)

Der Katalog scheint funktionsfähig und kann nun freigegeben werden. Der einfachste Weg, einen Intake-Katalog freizugeben, besteht darin, ihn an einem Ort abzulegen, an dem er von eurer Zielgruppe gelesen werden kann. In diesem Tutorial, das in einem Git-Repo gespeichert ist, kann dies die URL der Datei im Repo sein. Alles, was ihr für eure Benutzer freigeben müsst, ist die URL des Katalogs. Ihr könnt dies selbst ausprobieren mit:

In [11]:
cat = intake.open_catalog(
    "https://raw.githubusercontent.com/veit/Python4DataScience-de/main/docs/data-processing/intake/sea.yaml"
)

In [12]:
cat.sea_ice.read().head()

,Time,Arctic,Antarctica
0,1990M01,12.72,3.27
1,1990M02,13.33,2.15
2,1990M03,13.44,2.71
3,1990M04,12.16,5.10
4,1990M05,10.84,7.37


<div class="alert alert-block alert-info">

**Hinweis**

Dieser Katalog ist auch wieder eine DataSource-Instanz, d.h., dass ihr aus anderen Katalogen darauf verweisen und so eine Hierarchie von Datenquellen aufbauen könnt. Beispielsweise verfügt ihr über einen Stamm- oder Hauptkatalog, der auf mehrere andere Kataloge verweist, die jeweils Einträge eines bestimmten Typs enthalten. und das Ganze kann z.B. mit [Intake-GUI](https://www.python4data.science/de/latest/data-processing/intake/gui.html) durchsucht werden. Auf diese Weise hat die Datenerfassung insgesamt eine Struktur, die das Navigieren zum richtigen Datensatz vereinfacht. Ihr könnt sogar separate Hierarchien haben, die auf dieselben Daten verweisen.
</div>

In [13]:
print(cat.yaml())

sources:
  sea:
    args:
      path: https://raw.githubusercontent.com/veit/Python4DataScience-de/main/docs/data-processing/intake/sea.yaml
    description: ''
    driver: intake.catalog.local.YAMLFileCatalog
    metadata: {}

